In [15]:
from random import shuffle
from csv import reader 
from sklearn import metrics
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

filename = 'dataset.csv'

n_folds =20
max_depth = 10
min_size = 5

In [2]:
def k_fold_cross_validation(items, randomize=False):
    k=n_folds
    if randomize:
        items = list(items)
        shuffle(items)
    slices = [items[i::k] for i in range(k)]
    return slices


def data(fname):
    X,Z=list(),list()
    with open(fname,'r') as f:
        contents = f.readlines()
        Z+= contents
    for i in Z:
        my_list = i.strip("\n").strip("\r").split(",")
        X.append(my_list)
    try:
        A=[]
        for i in range(1,len(X)):
            Y = [float(j) for j in X[i]] 
            A.append(Y)
    except ValueError:
        print ("error on line",i)
    return A

In [3]:
def gini(groups,class_values):
    gini_value = 0.0
    for class_value in class_values: 
        for group in groups:
            number=0.0
            size = len(group)
            if size == 0:
                continue
            for row in group:
                if(row[-1]==class_value):
                    number+=1
            ratio= number/float(len(group))

            gini_value += (ratio* (1.0 - ratio))
    return gini_value
 


def calculate_accuracy(dataset):
    folds=k_fold_cross_validation(dataset)
    scores = list()
    for i in range(n_folds):
        fold_at_i = folds[i]
        training=list()
        for j in range(n_folds):
            if j!=i:
                training.append(folds[j])
        training = sum(training, [])
        testing = list()
        for row in fold_at_i:
            row_copy = list(row)
            row_copy[-1] = None
            testing.append(row_copy)
        predicted = decision_tree(training, testing)
        actual = [row[-1] for row in fold_at_i]
        correct_values = 0
        for i in range(len(actual)):
            if (actual[i] == predicted[i]):
                correct_values += 1
        accuracy= correct_values / float(len(actual)) * 100.0
        scores.append(accuracy)
    return scores

In [4]:
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right
 

def splitting(dataset):
    class_values = list(set(row[-1] for row in dataset))
    node_index=999
    node_value=999
    node_score=999
    node_groups =None
    for index in range(len(dataset[0])-1):
        count=0.0
        count_row=0.0
        for row in dataset: 
            count+=1
            count_row=row[index]+count_row
        count_row=count_row/count
        groups=test_split(index,count_row,dataset)
        gini_value=gini(groups,class_values)
        if gini_value < node_score:
                node_index, node_value, node_score, node_groups = index, row[index], gini_value, groups
    return {'i':node_index, 'value':node_value, 'div':node_groups} 


In [5]:
def terminal_node(group):
    outcomes = [row[-1] for row in group]
    classify1=0
    classify2=0
    classify3=0
    for out in outcomes:
        if int(out)==1:
            classify1+=1
        if int(out)==2:
            classify2+=1
        if int(out)==3:
            classify3+=1
        if(classify1 > classify2 and classify1 > classify3):
            return 1
        elif classify2 > classify3:
            return 2
        else:
            return 3


In [6]:
def node_split(node,depth):
    left, right = node['div']
    del(node['div'])

    if not left or not right:
        node['left'] = node['right'] = terminal_node(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = terminal_node(left), terminal_node(right)
        return

    if len(left) <= min_size:
        node['left'] = terminal_node(left)
    else:
        node['left'] = splitting(left)
        node_split(node['left'], depth+1)
    if len(right) <= min_size:
        node['right'] = terminal_node(right)
    else:
        node['right'] = splitting(right)
        node_split(node['right'], depth+1)


In [7]:
def output_from_tree(node, row):
    if row[node['i']] < node['value']:
        if (type(node['left'])== dict):
            return output_from_tree(node['left'], row)
        else:
            return node['left']
    else:
        if (type(node['right'])== dict):
            return output_from_tree(node['right'], row)
        else:
            return node['right']
 
# Classification and Regression Tree Algorithm
def decision_tree(train, test):
    root= splitting(train)
    node_split(root,1)
    predictions = list()
    for row in test:
        prediction = output_from_tree(root, row)
        predictions.append(prediction)
    return(predictions)


dataset = data(filename)
scores = calculate_accuracy(dataset)

In [8]:
accuracy=(sum(scores)/(len(scores)))
print("DT", accuracy)

DT 88.96253602305475


In [9]:
def ac_pred(dataset):
    folds=k_fold_cross_validation(dataset)
    scores = list()
    for i in range(n_folds):
        fold_at_i = folds[i]
        training=list()
        for j in range(n_folds):
            if j!=i:
                training.append(folds[j])
        training = sum(training, [])
        testing = list()
        for row in fold_at_i:
            row_copy = list(row)
            row_copy[-1] = None
            testing.append(row_copy)
        predicted = decision_tree(training, testing)
        actual = [row[-1] for row in fold_at_i]
        
    return actual,predicted

In [10]:
actual,predicted=ac_pred(dataset)

In [16]:
print('Precision:', metrics.precision_score(actual, predicted,average="weighted")*100)
print('Recall:', metrics.recall_score(actual, predicted,average="weighted")*100)
print('F1-Score:', metrics.f1_score(actual, predicted,average="weighted")*100)

Precision: 79.81130978581335
Recall: 89.33717579250721
F1-Score: 84.30601064133099


In [12]:
ACCURACY = metrics.accuracy_score(actual, predicted)*100
PRECISION = metrics.precision_score(actual, predicted,average="weighted")*100
RECALL = metrics.recall_score(actual, predicted,average="weighted")*100
F1_SCORE = metrics.recall_score(actual, predicted,average="weighted")*100

In [13]:
data=pd.DataFrame([['Decision Tree',ACCURACY,PRECISION,RECALL,F1_SCORE]],columns=['Model','ACCURACY','PRECISION','RECALL','F1-SCORE'])
data.to_csv('scores_dt.csv',index=False)

In [14]:
data

Model   ACCURACY  PRECISION     RECALL   F1-SCORE
0  Decision Tree  89.337176   79.81131  89.337176  89.337176